In [30]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import spacy
from spacy import displacy

In [31]:
url = "https://www.nbr.org/publication/enhancing-clean-energy-cooperation-in-the-indo-pacific/"
response = requests.get(url)

In [32]:
soup=BeautifulSoup(response.content, "html.parser")

In [33]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Enhancing Clean Energy Cooperation in the Indo-Pacific | The National Bureau of Asian Research (NBR)
  </title>
  <link href="https://www.nbr.org/xmlrpc.php" rel="pingback"/>
  <link href="https://www.nbr.org/wp-content/themes/nbr-theme/build/css/main.css" rel="stylesheet" type="text/css"/>
  <link href="https://www.nbr.org/wp-content/themes/nbr-theme/style.css" media="screen" rel="stylesheet" type="text/css"/>
  <script crossorigin="anonymous" defer="" integrity="sha384-3yBLeJ4waqGSAf4A8pjZ13UF7GuhgbdKnBQvIp/TkWoXtQbtwjlIPNjkDRJ46UCn" src="https://pro.fontawesome.com/releases/v5.5.0/js/all.js">
  </script>
  <meta content="max-image-preview:large" name="robots"/>
  <link href="//code.jquery.com" rel="dns-prefetch"/>
  <

In [34]:
main_content = soup.find('main')
for element in main_content(['a', 'em']):
    element.decompose()

In [35]:
text_list = []
stop_text = "James Bowen is a Policy Fellow at the Perth USAsia Centre."
for paragraph in main_content.find_all('p'):
    paragraph_text = paragraph.get_text(separator=' ', strip=False)

    if stop_text in paragraph_text:
        text_list.append(paragraph_text)
        break
    text_list.append(paragraph_text)

In [36]:
df = pd.DataFrame(text_list, columns=["Paragraph Text"])
df.to_csv('02_paragraphs.csv', index=False)

In [37]:
df

,Paragraph Text
0,James Bowen argues that clean energy cooperati...
1,Ensuring a rapid global transition to clean en...
2,The Intergovernmental Panel on Climate Change’...
3,Regional coordination of clean energy commitme...
4,Existing Indo-Pacific clean energy relationshi...
5,The current clean energy engagement of the Uni...
6,Key U.S. regional ally Australia is prominentl...
7,"Japan, meanwhile, has emerged as the dominant ..."
8,"Like Japan, South Korea is a major regional hy..."
9,The potential game-changer for Indo-Pacific cl...


In [38]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [39]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/loogyee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/loogyee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
nlp = spacy.load("en_core_web_sm")

In [42]:
df = pd.read_csv('02_paragraphs.csv')

In [43]:
# Preprocessing 
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the entire text
    text = text.lower()
    
    # Tokenize and lemmatize
    lemmatized_words = []
    for word in text.split():  # Split the text into words
        lemmatized_words.append(lemmatizer.lemmatize(word))  # Lemmatize each word
    
    # Join lemmatized words back into a single string
    cleaned_text = ' '.join(lemmatized_words)
    
    return cleaned_text

In [44]:
df['Processed Text'] = df['Paragraph Text'].apply(preprocess_text)

In [45]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [46]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [47]:
relationship_keywords = {'collaboration', 'cooperation', 'partner', 'engage', 
                         'support', 'fund', 'finance', 'invest', 'develop', 
                         'supply', 'promote', 'transition', 'export', 'agreement', 
                         'interest', 'share', 'enhance', 'effort', 'import'}

In [48]:
def extract_entities_and_relationships(text):
    doc = nlp(text)  # Process text with spaCy
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['ORG', 'GPE']]  # Extract only ORG and GPE entities

    # Find relationships based on keywords
    relationships = []
    for token in doc:
        if token.text in relationship_keywords:  # Check if the token matches any relationship keyword
            subject = [w for w in token.children if w.dep_ == "nsubj"]  # Find the subject
            object_ = [w for w in token.children if w.dep_ == "dobj"]  # Find the direct object
            if subject and object_:
                relationships.append((subject[0].text, token.text, object_[0].text))  # (subject, relationship keyword, object)

    return entities, relationships

In [49]:
df['Entities'], df['Relationships'] = zip(*df['Processed Text'].apply(extract_entities_and_relationships))

In [50]:
df[['Processed Text', 'Entities', 'Relationships']]

,Processed Text,Entities,Relationships
0,james bowen argues that clean energy cooperati...,"[(the united state, ORG)]",[]
1,ensuring a rapid global transition to clean en...,"[(un, ORG), (the united state, ORG)]",[]
2,the intergovernmental panel on climate change’...,[],[]
3,regional coordination of clean energy commitme...,[(national clean energy transition require mas...,[]
4,existing indo-pacific clean energy relationshi...,"[(china, GPE), (china, GPE), (china, GPE), (be...",[]
5,the current clean energy engagement of the uni...,"[(the united state, ORG)]",[]
6,key u.s. regional ally australia is prominentl...,"[(u.s., GPE), (australia, GPE), (australia, GP...",[]
7,"japan, meanwhile, ha emerged a the dominant su...","[(japan, GPE), (australia, GPE), (tokyo, GPE),...","[(investment, enhance, security)]"
8,"like japan, south korea is a major regional hy...","[(japan, GPE), (south korea, GPE), (seoul, GPE...",[]
9,the potential game-changer for indo-pacific cl...,"[(india, GPE), (china, GPE), (india, GPE), (in...",[]


In [51]:
for index, row in df.iterrows():
    print(f"\nParagraph {index + 1}:")
    print(row['Paragraph Text'])

    # Display dependency parsing
    doc = nlp(row['Processed Text'])
    sentence_spans = list(doc.sents)
    displacy.render(sentence_spans, style='dep', jupyter=True, options={'compact': True})
    
    # Display named entities
    displacy.render(sentence_spans, style="ent", jupyter=True)

    # Print extracted entities and relationships
    print("Entities:", row['Entities'])


Paragraph 1:
James Bowen argues that clean energy cooperation would be a win for both the climate and stability of the long-standing Indo-Pacific order and urges the United States and other advanced regional economies to revive the spirit of common cause that followed past energy crises.


Entities: [('the united state', 'ORG')]

Paragraph 2:
Ensuring a rapid global transition to clean energy systems is the overriding priority of international climate action. Cross-border cooperation in this space is critical, yet it has proved exceedingly difficult at the all-inclusive UN-led level. Smaller avenues of parallel activity could ultimately deliver more meaningful progress. Collaborative efforts that simultaneously allow space for the advancement of national economic and strategic positions are a prominent feature of current Indo-Pacific relations. They merit sustained commitment from the United States and its regional allies and partners, particularly at a time of upheaval in global energy markets.


/Users/loogyee/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Entities: [('un', 'ORG'), ('the united state', 'ORG')]

Paragraph 3:
The Intergovernmental Panel on Climate Change’s April 2022 report noted significant “geophysical, environmental-ecological, technological, economic and, especially, institutional and socio-cultural” barriers to an otherwise feasible transition to clean energy and broader economic systems.  All these obstacles are evident to typically high degrees in Indo-Pacific countries, whose dominant emissions pathways guarantee the region’s ongoing centrality in global decarbonization efforts.


Entities: []

Paragraph 4:
Regional coordination of clean energy commitments is essential to breaking down barriers for four key reasons. First, many Indo-Pacific countries will struggle to meet their decarbonization goals without access to international trade in clean electricity and other clean energy carriers, such as hydrogen. Second, the changing economics of clean energy versus fossil fuels will reorder trade for some commodities and goods, with value-adding shifts to new low-cost markets. Third, clean energy systems generate a much higher demand for unevenly distributed “critical minerals”—such as lithium, cobalt, nickel, and rare earth minerals—than do hydrocarbon equivalents. Fourth, national clean energy transitions require mass deployment of novel technologies and infrastructure, which calls for significant external financial and technical assistance for developing economies.


...

Entities: [('russia', 'GPE'), ('the united states', 'GPE'), ('u.s.', 'GPE')]

Paragraph 16:
There is a need to ensure that short-term geopolitical exigencies do not derail longer-term imperatives, not least in the Indo-Pacific. Indeed, there is an even more pressing requirement for the United States and its partners to cooperate on meeting regional decarbonization needs than existed prior to the Ukraine invasion. Such cooperation is now also critical to heading off any attempts by Russia to transfer its European fossil fuel interdependencies to the energy-hungry Indo-Pacific economies. This would be a disaster for not only the climate but the future geopolitical orientation of the region.


Entities: [('the united state', 'ORG'), ('russia', 'GPE')]

Paragraph 17:
The United States and other advanced regional economies should revive the spirit of common cause that followed past energy crises. The legacy of the 1973 Arab oil embargo, most notably, included formation of the IEA and its various commitments to collective energy security. Washington and its allies should in turn recognize the new realities of the growing decarbonization imperative and the shift of global energy interactions to the Indo-Pacific. They should commit sustained resources to working alongside one another and with regional partners to connect clean energy needs and abilities.


Entities: [('the united state', 'ORG'), ('washington', 'GPE')]

Paragraph 18:
The Biden administration’s Indo-Pacific Economic Framework would be an ideal instrument for shepherding progress. Regional interlocutors have already expressed strong interest in the new framework’s potential offerings in this space.  Seizing on the imperative for clean energy cooperation would be a win for both the climate and stability of the long-standing Indo-Pacific order.


Entities: []

Paragraph 19:
James Bowen is a Policy Fellow at the Perth USAsia Centre.


Entities: [('the perth usasia centre', 'ORG')]


In [52]:
entity1_list = []
relationship_list = []
entity2_list = []
date_list = []
money_list = []

In [53]:
# Iterate through each paragraph
for index, row in df.iterrows():
    doc = nlp(row['Paragraph Text'])  # Process the paragraph with spaCy

    for sent in doc.sents:
        # Extract named entities (countries, organizations)
        entities = []
        money_in_sentence = []

        for ent in sent.ents:
            if ent.label_ == 'Initiative' in ent.text:
                entities.append(ent._replace(label_='POL'))  # Change label for specific case
            elif ent.label_ in ['GPE', 'ORG', 'POL', 'MONEY']:
                entities.append(ent)

            # Collect monetary amounts
            if ent.label_ == "MONEY":
                money_in_sentence.append(ent.text)
        
        # Look for verbs or specific relationship nouns in the sentence
        verbs = [token for token in sent if token.pos_ == "VERB"]
        keywords_in_sentence = [token.text for token in sent if token.lemma_ in relationship_keywords]
        
        # Extract dates
        dates = [ent.text for ent in sent.ents if ent.label_ == "DATE"]

        # If we find two entities and a relationship keyword, we assume a relationship
        if len(entities) >= 2 and keywords_in_sentence:
            entity1_list.append(entities[0].text)

            # Concatenate the relationship noun and verb (if available) into one string
            relationship_info = keywords_in_sentence[0]
            if verbs:
                relationship_info += " (verb: " + verbs[0].lemma_ + ")"

            relationship_list.append(relationship_info)  # Store concatenated info
            entity2_list.append(entities[1].text)
            date_list.append(dates[0] if dates else "n/a")
            money_list.append(money_in_sentence[0] if money_in_sentence else "n/a")

In [54]:
# DataFrame
extracted_data = pd.DataFrame({
    "Entity1": entity1_list,
    "Relationship": relationship_list,
    "Entity2": entity2_list,
    "Date": date_list,
    "Money": money_list
})

In [55]:
extracted_data

,Entity1,Relationship,Entity2,Date,Money
0,Indo-Pacific,cooperation (verb: argue),the United States,n/a,n/a
1,the United States,partners,Indo-Pacific,n/a,n/a
2,Sun Cable,funded (verb: fund),Singapore,n/a,n/a
3,Australia,develop (verb: work),Brunei,n/a,n/a
4,Tokyo,efforts (verb: mirror),LNG,n/a,n/a
5,Indo-Pacific,cooperation,India,n/a,n/a
6,India,export (verb: reveal),National Hydrogen Mission,2021,n/a
7,New Delhi,agreement (verb: sign),Canberra,2022,n/a
8,The United States,supporting (verb: play),Indo-Pacific,n/a,n/a
9,U.S.,export (verb: commit),U.S.,March 2022,n/a


In [56]:
# Export to csv
extracted_data.to_csv('02_relationship.csv', index=False)